# Introduçãos às CNNs no PyTorch e ilustrações dos resultados obtidos para cada camada da DNN

## Objetivos

Este notebook contém exemplo numérico de uma rede com uma camada convolucional e uma camada densa. A camada convolucional possui ativação reLU e max-pooling. Já a camada densa possui uma única saída com ativação sigmóide.

Com esse exemplo, aprende-se a:
- construir a rede utilizando tanto o modelo sequencial como o via API
- inicializar os pesos e biases da rede convolucional e da rede densa
- visualizar os dados intermediários da rede

## Importação dos Módulos

In [21]:
import numpy as np
import os

import torch
import torch.nn as nn
import torch.functional as F

from collections import OrderedDict

np.set_printoptions(precision=3) # ponto flutuante com 3 casas para facilitar a impressão

In [2]:
# verifica se a GPU está disponível
use_gpu = torch.cuda.is_available()
print("Usando GPU:", use_gpu)

Usando GPU: True


## Definição da Rede, camadas convolucionais e densas

Uma CNN pode ser construída de dois modos no PyTorch. Ela pode ser implementada usando

* Modelo Sequential: modelo mais simpes, utilizado quando a rede é composto de camadas uma após a outra.
* Modelo subclasse de "Module": é o modelo mais geral, baseado em instâncias de camadas de podem ser montadas da forma como se desejar.

### Definição de camadas no PyTorch

Em redes neurais, uma camada é usualmente um neurônio, que inclui a soma de multiplicação de
pesos ou convolução e uma ativação usualmente não linear. Podemos dizer que o max-pooling 
também faz parte da camada.

A rede que iremos utilizar neste exemplo possui 2 camadas: uma convolucional e outra densa.
A camada convolucional terá ativação reLU e um max-pooling, já a camada densa terá uma
ativação sigmóide.

### Rede a ser implementada

<img src='../figures/RedeIntroKeras.png', width=600pt>

### Implementação do modelo

In [6]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        # camada convolucional
        self.conv = nn.Conv2d(1, 3, (2, 2), padding=0, bias=True)
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d((2, 2))
        
        # camada densa
        self.dense = nn.Linear(4*5, 1, bias=True)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        # passa os dados pela camada convolucional
        x = self.conv(x)
        x = self.relu(x)
        x = self.max_pool(x)
        
        # faz o flatten dos dados
        x = x.view(-1, 4*5)
        
        # passa os dados pela camada densa
        x = self.dense(x)
        x = self.sigmoid(x)
        
        return x   

In [7]:
model = Model()
print(model)

Model (
  (conv): Conv2d(1, 3, kernel_size=(2, 2), stride=(1, 1))
  (relu): ReLU ()
  (max_pool): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (dense): Linear (20 -> 1)
  (sigmoid): Sigmoid ()
)


### Outra forma de implementação do modelo

In [23]:
class Model2(nn.Module):
    def __init__(self):
        super(Model2, self).__init__()
        
        # camada convolucional
        self.layer1 = nn.Sequential(OrderedDict([
            ('conv', nn.Conv2d(1, 3, (2, 2), padding=0, bias=True)),
            ('relu', nn.ReLU()),
            ('max_pool', nn.MaxPool2d((2, 2)))
        ]))
        
        # camada densa
        self.layer2 = nn.Sequential(OrderedDict([
            ('dense', nn.Linear(4*5, 1, bias=True)),
            ('sigmoid', nn.Sigmoid())
        ]))
        
    def forward(self, x):
        # passa os dados pela camada convolucional
        x = self.layer1(x)
        # faz o flatten dos dados
        x = x.view(-1, 4*5)
        # passa os dados pela camada densa
        x = self.layer2(x) 
        
        return x   

In [24]:
model2 = Model2()
print(model2)

Model2 (
  (layer1): Sequential (
    (conv): Conv2d(1, 3, kernel_size=(2, 2), stride=(1, 1))
    (relu): ReLU ()
    (max_pool): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  )
  (layer2): Sequential (
    (dense): Linear (20 -> 1)
    (sigmoid): Sigmoid ()
  )
)


In [4]:
        
def build_model():
    
    conv = nn.Sequential([
        nn.Conv2d(input_channels, n_filters, kernel_shape, padding=0, bias=use_bias),
        nn.ReLU(),
        nn.MaxPool2d(kernel_shape),
        
        nn.Linear()
    ])
    
    dense = Seque
    
    x = conv(x)
    x = flatten(x)
    x = dense(x)
    
    model.add(Conv2D(n_filters, # número de filtros
                     kernel_shape, # tamanho do kernel
                     name = 'conv_1', 
                     padding = 'valid',
                     use_bias = use_bias,
                     input_shape = input_shape,
                     data_format = "channels_first"))
    model.add(Activation('relu', name = 'relu_1'))
    model.add(MaxPooling2D(pool_size = (2, 2), 
                           data_format = "channels_first",
                           name = 'max_pool_1'))
    
    model.add(Flatten(name = 'flat'))
    
    model.add(Dense(1, 
                    use_bias = use_bias,
                    name = 'dense_2'))
    model.add(Activation('sigmoid', name = 'sigmoid_2'))
    
    return model

### Implementação com *subclasse de Module*

In [3]:



def build_model_API():
    
    input_shape = (1,5,6) # (canais, linhas, colunas)
    n_filters = 3
    kernel_shape = (2,2)
    use_bias = True
    inputs = Input(input_shape, name = 'input')
    conv1 = Conv2D(n_filters, 
                   kernel_shape, 
                   name = 'conv1', 
                   padding = 'valid',
                   use_bias = use_bias,
                   data_format = "channels_first")(inputs) 
    actv1 = Activation('relu', name = 'relu1')(conv1)
    pool1 = MaxPooling2D(pool_size=(2,2),
                         data_format = "channels_first",
                         name = 'max_pool1')(actv1) 
    flat = Flatten(name = 'flat')(pool1)
    dense1 = Dense(1, 
                   use_bias = use_bias,
                   name = 'dense2')(flat)
    out = Activation('sigmoid', name = 'sigmoid2')(dense1)

    model = Model(inputs=inputs, outputs=out)
    
    return model

# Configurando a rede neural com convolução e densa

## Definição dos parâmetros da rede

### Imagem de Entrada: 5 linhas e 6 colunas

In [8]:
X = np.array([[[[0,0,0,0,0,0],
                [0,0,1,0,0,0],
                [0,0,0,0,0,0],
                [0,0,0,0,-1,0],
                [0,0,0,0,0,0]]]])

(n_samples, n_channels, img_height, img_width) = X.shape
input_shape = (n_channels, img_height, img_width)
print('X.shape=',X.shape)
print('X:\n',X)

X.shape= (1, 1, 5, 6)
X:
 [[[[ 0  0  0  0  0  0]
   [ 0  0  1  0  0  0]
   [ 0  0  0  0  0  0]
   [ 0  0  0  0 -1  0]
   [ 0  0  0  0  0  0]]]]


### Kernel da convolução

In [10]:
# número de filtros
n_filters = 3    

# comprimento e largura dos filtros
k_height = k_width = 2 
kernel_shape = (k_height,k_width)

Win = np.array([[[1,2],
                 [3,4]],
                [[5,6],
                 [7,8]],
                [[9,10],
                 [11,12]]]).reshape(n_filters,1,k_height,k_width)

print('Win.shape (n_filters, n_channels, k_height, k_width):',Win.shape)
print('Win: (Pesos camada convolucional, formato mais fácil de entender)\n',Win)

W_conv = Win.transpose(2,3,1,0)
# Para deixar no formato do Keras: (k_height,k_width,n_channels,n_filters)
print('Wconv.shape (k_height,k_width,n_channels,n_filters):',W_conv.shape)
print('Wconv: (Pesos camada convolucional, formato do Keras)\n',W_conv)


Win.shape (n_filters, n_channels, k_height, k_width): (3, 1, 2, 2)
Win: (Pesos camada convolucional, formato mais fácil de entender)
 [[[[ 1  2]
   [ 3  4]]]


 [[[ 5  6]
   [ 7  8]]]


 [[[ 9 10]
   [11 12]]]]
Wconv.shape (k_height,k_width,n_channels,n_filters): (2, 2, 1, 3)
Wconv: (Pesos camada convolucional, formato do Keras)
 [[[[ 1  5  9]]

  [[ 2  6 10]]]


 [[[ 3  7 11]]

  [[ 4  8 12]]]]


### Bias da Convolução

In [11]:
# valor de bias
f_bias = 0.1     
bias_conv = np.arange(1,n_filters+1) * f_bias
print("Bias da convolução:",bias_conv)

Bias da convolução: [ 0.1  0.2  0.3]


### Pesos para a camada densa

In [12]:
# após o max pooling, são 3 imagens 2x2 = 12
W_dense = np.arange(12).reshape(12,1)
print("Pesos da camada densa:\n",W_dense)

Pesos da camada densa:
 [[ 0]
 [ 1]
 [ 2]
 [ 3]
 [ 4]
 [ 5]
 [ 6]
 [ 7]
 [ 8]
 [ 9]
 [10]
 [11]]


### Bias para a camada densa

In [13]:
bias_dense = np.ones(1) * f_bias
print("Bias da camanda densa:",bias_dense)

Bias da camanda densa: [ 0.1]


## Imprimindo os valores dos dados das camadas internas da rede

Como a rede é executada no Keras como *backend*, não existem variáveis de fácil acesso para
conseguir ver os dados nas camadas intermediárias da rede. Assim, o artifício para se conseguir
isso é criar várias redes, uma rede com apenas a primeira camada, e fazer a predição. O resultado
dessa predição são os dados após a primeira camada. Colocar a segunda camada e fazer uma nova predição, obtendo
os dados após a segunda camada e assim sucessivamente até fazer a predição da rede completa. Isso
é implementado no código a seguir:

In [14]:
model = Model()
#model = build_model_Sequential()
print(W_conv.shape)
print(bias_conv.shape)
print(W_dense.shape)
print(bias_dense.shape)
model.set_weights([W_conv, bias_conv, W_dense, bias_dense])

print('-'*30)
print("Número de camadas:", len(model.layers))
print('-'*30)

# Resultados para cada camada
i = 1
for layer in model.layers:
    intermediate_layer_model = Model(inputs=model.input,outputs=layer.output)
    intermediate_output = intermediate_layer_model.predict(X)
    print('-'*80)
    print("Saída da camada", i, ":", layer.name, "shape:", intermediate_output.shape)
    print('-'*80)
    print(intermediate_output)
    i+=1

NameError: name 'build_model_API' is not defined

## Sumário 

É sempre útil imprimir o sumário da rede. Ele contém informações de cada camada da rede, 
como nome, tipo de camada, shape do tensor na saída da camada e número de parâmetros.

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 1, 5, 6)           0         
_________________________________________________________________
conv1 (Conv2D)               (None, 3, 4, 5)           15        
_________________________________________________________________
relu1 (Activation)           (None, 3, 4, 5)           0         
_________________________________________________________________
max_pool1 (MaxPooling2D)     (None, 3, 2, 2)           0         
_________________________________________________________________
flat (Flatten)               (None, 12)                0         
_________________________________________________________________
dense2 (Dense)               (None, 1)                 13        
_________________________________________________________________
sigmoid2 (Activation)        (None, 1)                 0         
Total para

# Sugestões de atividades

1. Executar o experimento nos dois modos de construção da rede, API e Sequential. 
   Existe alguma diferença?
2. Calcular o número de parâmetros da rede. Confirmar com o sumário apresentado.
3. Retirar o bias e recalcular o número de parâmetros a serem treinados
4. A rede foi projetada para aceitar entrada com shape (1,5,6). Mudar a rede para aceitar:
   a) entrada com shape (1,6,6)
   b) entrada com shape (3,6,6)
5. Inserir mais uma camada convolucional com 4 filtros de saída e kernel (3,3)


# Referências

- [Keras Documentation](https://keras.io/)


# Aprendizados com este notebook
